## Problem Set 7

### Joseph Denby

In [107]:
import pandas as pd
import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics 
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm


In [108]:
auto = pd.read_csv('../../../ProblemSets/PS7/Auto.csv', na_values='?')
auto = auto.dropna()

In [109]:
auto['mpg_high'] = auto['mpg'].apply(lambda x: int(x>=np.median(auto['mpg'])))
auto['const'] = 1

In [110]:
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name,mpg_high,const
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu,0,1
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320,0,1
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite,0,1
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst,0,1
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino,0,1


(a) Use `sklearn.linear model.LogisticRegression` to fit a logistic model of `mpg_high` on features number of cylinders (`cyl`), engine displacement (`dspl`), horsepower (`hpwr`), vehicle weight (`wgt`), acceleration (`accl`), vehicle year (`yr`), vehicle origin (`orgn`). Make sure to include a constant term. Fit the model using $k$-fold cross validation with $k = 4$ folds.
Report the MSE of the model as the average MSE across the $k = 4$ test sets, and report the error rates for each category of `mpg_high` as the average error rate for that category across the $k = 4$ test sets.


In [111]:
X = auto[['const', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
y = auto['mpg_high'].values

In [112]:
k = 4
kf_log = KFold(n_splits=k, shuffle=True, random_state=15)
kf_log.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
true_MSE_kf = np.zeros(k)
false_MSE_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_log.split(X):
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    true_vals = np.asarray(true_vals)
    true_preds = np.asarray(true_preds)
    false_vals = np.asarray(false_vals)
    false_preds = np.asarray(false_preds)
    true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
     
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = True is', true_MSE_kf[k_ind])
    print('MSE for mpg_high = False is', false_MSE_kf[k_ind])

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('error rate for mpg_high = True', true_MSE,
      'error rate for mpg_high = False', false_MSE)

k index= 0
MSE for test set 0  is 0.09183673469387756
MSE for mpg_high = True is 0.13043478260869565
MSE for mpg_high = False is 0.057692307692307696
k index= 1
MSE for test set 1  is 0.10204081632653061
MSE for mpg_high = True is 0.08163265306122448
MSE for mpg_high = False is 0.12244897959183673
k index= 2
MSE for test set 2  is 0.1326530612244898
MSE for mpg_high = True is 0.11538461538461539
MSE for mpg_high = False is 0.15217391304347827
k index= 3
MSE for test set 3  is 0.10204081632653061
MSE for mpg_high = True is 0.14545454545454545
MSE for mpg_high = False is 0.046511627906976744
test estimate MSE k-fold= 0.10714285714285715 test estimate MSE standard err= 0.015306122448979593
error rate for mpg_high = True 0.11822664912727025 error rate for mpg_high = False 0.09470670705864986


(b) Use `sklearn.ensemble.RandomForestClassifier` to fit a random forest model of `mpg_high` on `max_features=2` out of the seven possible features used in part (a). Set `n_estimators=20`, set `bootstrap=True`, set `oob_score=True`, and set `random_state=25`. Report the MSE of the random forest model as the MSE from the `.oob_prediction_` object, and report the error rates for each category of `mpg_high` from the `.oob_prediction_`object.

In [113]:
auto_tree = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,
                                  oob_score=True, random_state=25)
auto_tree.fit(X, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [114]:
rnd = np.vectorize(lambda x: int(x >= .5))
ypred = rnd(auto_tree.oob_decision_function_[:,1])

In [115]:
MSE = mean_squared_error(y, ypred)
print('MSE=', MSE)

MSE= 0.07908163265306123


In [116]:
autotreedf = pd.DataFrame({'y':y,'ypred':ypred})
autotreedf['MSE'] = autotreedf.apply(lambda x: (x['y']-x['ypred'])**2, axis=1)
print('MSE for mpg_high = True:', autotreedf.groupby(y)['MSE'].mean()[1])
print('MSE for mpg_high = False:', autotreedf.groupby(y)['MSE'].mean()[0])

MSE for mpg_high = True: 0.07142857142857142
MSE for mpg_high = False: 0.08673469387755102


(c) Use `sklearn.svm.SVC` to fit a support vector machines model of `mpg_high` with a Gaussian radial basis function kernel `kernel=’rbf’` on the seven features used in part (a). Set the penalty parameter to `C=1` and set `gamma=0.2`. Fit the model using $k$-fold cross validation with $k = 4$ folds exactly as in part (a). Report the MSE of the model as the average MSE across the $k = 4$ test sets, and report the error rates for each category of `mpg_high` as the average error rate for that category across the $k = 4$ test sets.


In [129]:
k = 4
kf_svm = KFold(n_splits=k, shuffle=True, random_state=15)
kf_svm.get_n_splits(X)

MSE_vec_kf = np.zeros(k)
true_MSE_kf = np.zeros(k)
false_MSE_kf = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf_log.split(X):
    print('k index=', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    clf = svm.SVC(kernel='rbf', C=1, gamma=.2)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    
    true_vals = []
    true_preds = []
    false_vals = []
    false_preds = []
    for i, p in enumerate(y_pred):
        if p == 1:
            true_preds.append(p)
            true_vals.append(y_test[i])
        else:
            false_preds.append(p)
            false_vals.append(y_test[i])
    
    if true_vals and true_preds:
        true_vals = np.asarray(true_vals)
        true_preds = np.asarray(true_preds)
        
        true_MSE_kf[k_ind] = ((true_vals - true_preds) ** 2).mean()
    else:
        true_MSE_kf[k_ind] = 0.0
    
    if false_vals and false_preds:
        false_preds = np.asarray(false_preds)
        false_vals = np.asarray(false_vals)
        
        false_MSE_kf[k_ind] = ((false_vals - false_preds) ** 2).mean()
    else:
        false_MSE_kf[k_ind] = 0.0
            
     
    print('MSE for test set', k_ind, ' is', MSE_vec_kf[k_ind])
    print('MSE for mpg_high = True is', true_MSE_kf[k_ind])
    print('MSE for mpg_high = False is', false_MSE_kf[k_ind])

    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)
true_MSE = true_MSE_kf.mean()
false_MSE = false_MSE_kf.mean()
print('error rate for mpg_high = True', true_MSE,
      'error rate for mpg_high = False', false_MSE)

k index= 0
MSE for test set 0  is 0.5408163265306123
MSE for mpg_high = True is 0.5520833333333334
MSE for mpg_high = False is 0.0
k index= 1
MSE for test set 1  is 0.5204081632653061
MSE for mpg_high = True is 0.0
MSE for mpg_high = False is 0.5204081632653061
k index= 2
MSE for test set 2  is 0.5204081632653061
MSE for mpg_high = True is 0.0
MSE for mpg_high = False is 0.53125
k index= 3
MSE for test set 3  is 0.4489795918367347
MSE for mpg_high = True is 0.0
MSE for mpg_high = False is 0.4731182795698925
test estimate MSE k-fold= 0.5076530612244898 test estimate MSE standard err= 0.03488467941626875
error rate for mpg_high = True 0.13802083333333334 error rate for mpg_high = False 0.38119411070879966


(d) Which of the above three models do you think is the best predictor of `mpg_high`? Why?

Comparing each model's aggregate MSE, the `RandomForestClassifier` is the most accurate model. This model has the lowest aggregate MSE, as well as the lowest MSE values for each level of `mpg_high`. The `LogisticRegression` model has comparable MSE values, while the `svm.SVC` model is far less accurate, with aggregate MSE values exceeding .5 (indicating floor performance on a binary classificiation task.)